In [216]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Set a random seed for reproducibility
random.seed(42)

# Generate a more realistic sample dataset
def generate_realistic_data(num_rows=500):
    data = []

    for i in range(num_rows):
        start_date = datetime(2020, 1, 1) + timedelta(days=random.randint(0, 365))
        
        # Land size influences project duration
        land_size = random.uniform(800, 5000)
        project_duration = int(max(30, land_size / 100))  # Assuming 100 sqm requires 1 day
        
        end_date = start_date + timedelta(days=project_duration)

        num_floors = random.randint(1, 10)
        
        # Establish relationships to ensure practical values
        tons_cement = max(5, min(land_size / 10, num_floors * 3.5))  # Adjusted cement quantity
        
        labor_required = max(5, min(num_floors * 5, 150))

        # Cost per unit for different materials
        cost_per_ton_cement = 8000
        cost_per_brick = 6
        cost_per_ton_iron = 50000
        cost_per_ton_sand = 7500
        # Calculate additional columns
        total_budget = (tons_cement * cost_per_ton_cement +
                        labor_required * 500)  # Assuming labor cost per worker is 500

        bricks_per_sqft = 15  # Adjust as needed
        num_bricks = int(land_size * bricks_per_sqft)

        tons_iron = max(0, min(land_size / 100, num_floors * 2))  # Assuming 1 floor requires 2 tons of iron
        sand_ton=tons_cement*2.5

        total_cost = (num_bricks * cost_per_brick +
                      tons_iron * cost_per_ton_iron +
                      total_budget+sand_ton*cost_per_ton_sand )

        data.append({
            'Project_ID': f'P-{i + 1}',
            'Land_Size_Sqm': round(land_size, 2),
            'Num_Floors': num_floors,
            'Tons_Cement': round(tons_cement, 2),
            'Labor_Required': labor_required,
            'Start_Date': start_date,
            'End_Date': end_date,
            'Num_Bricks': num_bricks,
            'Tons_Iron': round(tons_iron, 2),
            'Tons_Sand':sand_ton,
            'Total_Cost': total_cost,
        })

    return data


dataset = pd.DataFrame(generate_realistic_data())

csv_file_path = 'C:/Users/sriyo/Desktop/Bob\'s Esti-Mate/output.csv'

dataset.to_csv(csv_file_path, index=False)

print(dataset)


    Project_ID  Land_Size_Sqm  Num_Floors  Tons_Cement  Labor_Required  \
0          P-1        1267.59           5         17.5              25   
1          P-2        1737.49           2          7.0              10   
2          P-3        3910.80           9         31.5              45   
3          P-4        3280.07           1          5.0               5   
4          P-5        1193.52           4         14.0              20   
..         ...            ...         ...          ...             ...   
495      P-496        4429.84           6         21.0              30   
496      P-497        2647.00           8         28.0              40   
497      P-498        3451.82           5         17.5              25   
498      P-499        1035.92           9         31.5              45   
499      P-500        2103.71           8         28.0              40   

    Start_Date   End_Date  Num_Bricks  Tons_Iron  Tons_Sand    Total_Cost  
0   2020-11-23 2020-12-23       190

In [288]:
df=pd.read_csv('output.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Project_ID      500 non-null    object 
 1   Land_Size_Sqm   500 non-null    float64
 2   Num_Floors      500 non-null    int64  
 3   Tons_Cement     500 non-null    float64
 4   Labor_Required  500 non-null    int64  
 5   Start_Date      500 non-null    object 
 6   End_Date        500 non-null    object 
 7   Num_Bricks      500 non-null    int64  
 8   Tons_Iron       500 non-null    float64
 9   Tons_Sand       500 non-null    float64
 10  Total_Cost      500 non-null    float64
dtypes: float64(5), int64(3), object(3)
memory usage: 43.1+ KB


In [289]:
df

,Project_ID,Land_Size_Sqm,Num_Floors,Tons_Cement,Labor_Required,Start_Date,End_Date,Num_Bricks,Tons_Iron,Tons_Sand,Total_Cost
0,P-1,1267.59,5,17.5,25,2020-11-23,2020-12-23,19013,10.00,43.75,1.094703e+06
1,P-2,1737.49,2,7.0,10,2020-05-05,2020-06-04,26062,4.00,17.50,5.486220e+05
2,P-3,3910.80,9,31.5,45,2020-12-12,2021-01-20,58662,18.00,78.75,2.117097e+06
3,P-4,3280.07,1,5.0,5,2020-02-14,2020-03-17,49201,2.00,12.50,5.314560e+05
4,P-5,1193.52,4,14.0,20,2020-01-16,2020-02-15,17902,8.00,35.00,8.919120e+05
...,...,...,...,...,...,...,...,...,...,...,...
495,P-496,4429.84,6,21.0,30,2020-06-14,2020-07-28,66447,12.00,52.50,1.575432e+06
496,P-497,2647.00,8,28.0,40,2020-03-18,2020-04-17,39705,16.00,70.00,1.807230e+06
497,P-498,3451.82,5,17.5,25,2020-08-14,2020-09-17,51777,10.00,43.75,1.291287e+06
498,P-499,1035.92,9,31.5,45,2020-10-29,2020-11-28,15538,10.36,78.75,1.476314e+06


In [290]:
df['End_Date'] = pd.to_datetime(df['End_Date'], format="%Y-%m-%d")
df['Start_Date']=pd.to_datetime(df['Start_Date'],format="%Y-%m-%d")

In [291]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Project_ID      500 non-null    object        
 1   Land_Size_Sqm   500 non-null    float64       
 2   Num_Floors      500 non-null    int64         
 3   Tons_Cement     500 non-null    float64       
 4   Labor_Required  500 non-null    int64         
 5   Start_Date      500 non-null    datetime64[ns]
 6   End_Date        500 non-null    datetime64[ns]
 7   Num_Bricks      500 non-null    int64         
 8   Tons_Iron       500 non-null    float64       
 9   Tons_Sand       500 non-null    float64       
 10  Total_Cost      500 non-null    float64       
dtypes: datetime64[ns](2), float64(5), int64(3), object(1)
memory usage: 43.1+ KB


In [292]:
df['No.of days']=df['End_Date']-df['Start_Date']
df['Days'] = df['No.of days'].dt.days
df = df.drop('No.of days', axis=1)

In [293]:
df

,Project_ID,Land_Size_Sqm,Num_Floors,Tons_Cement,Labor_Required,Start_Date,End_Date,Num_Bricks,Tons_Iron,Tons_Sand,Total_Cost,Days
0,P-1,1267.59,5,17.5,25,2020-11-23,2020-12-23,19013,10.00,43.75,1.094703e+06,30
1,P-2,1737.49,2,7.0,10,2020-05-05,2020-06-04,26062,4.00,17.50,5.486220e+05,30
2,P-3,3910.80,9,31.5,45,2020-12-12,2021-01-20,58662,18.00,78.75,2.117097e+06,39
3,P-4,3280.07,1,5.0,5,2020-02-14,2020-03-17,49201,2.00,12.50,5.314560e+05,32
4,P-5,1193.52,4,14.0,20,2020-01-16,2020-02-15,17902,8.00,35.00,8.919120e+05,30
...,...,...,...,...,...,...,...,...,...,...,...,...
495,P-496,4429.84,6,21.0,30,2020-06-14,2020-07-28,66447,12.00,52.50,1.575432e+06,44
496,P-497,2647.00,8,28.0,40,2020-03-18,2020-04-17,39705,16.00,70.00,1.807230e+06,30
497,P-498,3451.82,5,17.5,25,2020-08-14,2020-09-17,51777,10.00,43.75,1.291287e+06,34
498,P-499,1035.92,9,31.5,45,2020-10-29,2020-11-28,15538,10.36,78.75,1.476314e+06,30


In [294]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Project_ID      500 non-null    object        
 1   Land_Size_Sqm   500 non-null    float64       
 2   Num_Floors      500 non-null    int64         
 3   Tons_Cement     500 non-null    float64       
 4   Labor_Required  500 non-null    int64         
 5   Start_Date      500 non-null    datetime64[ns]
 6   End_Date        500 non-null    datetime64[ns]
 7   Num_Bricks      500 non-null    int64         
 8   Tons_Iron       500 non-null    float64       
 9   Tons_Sand       500 non-null    float64       
 10  Total_Cost      500 non-null    float64       
 11  Days            500 non-null    int64         
dtypes: datetime64[ns](2), float64(5), int64(4), object(1)
memory usage: 47.0+ KB


In [295]:
x=df[['Land_Size_Sqm','Num_Floors']]
y=df.iloc[:, 2:]

In [296]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [297]:
x_train

,Land_Size_Sqm,Num_Floors
249,1708.82,8
433,2606.29,4
19,1967.49,6
322,4317.40,10
332,4852.26,3
...,...,...
106,2070.57,9
270,3408.31,10
348,993.16,1
435,2547.16,2


In [298]:
y_train=y_train.drop(['Start_Date','End_Date'],axis=1)
y_test=y_test.drop(['Start_Date','End_Date'],axis=1)

# XGBoost Model

In [299]:
import xgboost as xgb
model = xgb.XGBRegressor()

In [300]:
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [301]:
y_pred=model.predict(x_test)

In [302]:
y_test

,Num_Floors,Tons_Cement,Labor_Required,Num_Bricks,Tons_Iron,Tons_Sand,Total_Cost,Days
361,8,28.0,40,74960,16.0,70.00,2018760.0,49
73,6,21.0,30,37365,12.0,52.50,1400940.0,30
374,3,10.5,15,23065,6.0,26.25,726765.0,30
155,8,28.0,40,53246,16.0,70.00,1888476.0,35
104,6,21.0,30,67394,12.0,52.50,1581114.0,44
...,...,...,...,...,...,...,...,...
347,5,17.5,25,22480,10.0,43.75,1115505.0,30
86,1,5.0,5,30692,2.0,12.50,420402.0,30
75,7,24.5,35,40262,14.0,61.25,1614447.0,30
438,2,7.0,10,17747,4.0,17.50,498732.0,30


In [303]:
df_pred= pd.DataFrame(y_pred, columns=['Num_Floors', 'Tons_Cement','Labor_Required','Num_Bricks','Tons_Iron','Tons_Sand','Total_Cost','Days'])

In [304]:
df_pred['Num_Floors']=df_pred['Num_Floors'].round()
df_pred['Labor_Required']=df_pred['Labor_Required'].round()
df_pred['Num_Bricks']=df_pred['Num_Bricks'].round()

In [305]:
df_pred['Total_Cost'] = df_pred['Total_Cost'].astype(int)

In [306]:
csv_file_path = 'C:/Users/sriyo/Desktop/Bob\'s Esti-Mate/predvals.csv'

df_pred.to_csv(csv_file_path, index=False)



In [307]:
csv_file_path = 'C:/Users/sriyo/Desktop/Bob\'s Esti-Mate/testvals.csv'

y_test.to_csv(csv_file_path, index=False)

In [308]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, df_pred)*100
r2

96.55665231828917

# Random Forest Regression

In [311]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor()

In [312]:
model2.fit(x_train,y_train)

RandomForestRegressor()

In [313]:
y_pred_2=model2.predict(x_test)

In [315]:
df_pred_2= pd.DataFrame(y_pred_2, columns=['Num_Floors', 'Tons_Cement','Labor_Required','Num_Bricks','Tons_Iron','Tons_Sand','Total_Cost','Days'])

In [316]:
df_pred_2

,Num_Floors,Tons_Cement,Labor_Required,Num_Bricks,Tons_Iron,Tons_Sand,Total_Cost,Days
0,8.00,28.000,40.00,71818.76,16.00,70.0000,1999912.56,47.58
1,6.00,21.000,30.00,37916.14,12.00,52.5000,1404246.84,30.00
2,3.00,10.500,15.00,21619.19,6.00,26.2500,718090.14,30.00
3,7.99,27.965,39.95,53322.91,15.98,69.9125,1886976.21,34.99
4,6.00,21.000,30.00,67382.41,12.00,52.5000,1581044.46,44.14
...,...,...,...,...,...,...,...,...
95,5.00,17.500,25.00,22218.53,10.00,43.7500,1113936.18,30.00
96,1.01,5.020,5.05,31054.73,2.02,12.5500,424138.38,30.00
97,7.00,24.500,35.00,39636.25,14.00,61.2500,1610692.50,30.00
98,2.00,7.000,10.00,17840.21,4.00,17.5000,499291.26,30.00


In [317]:
y_test

,Num_Floors,Tons_Cement,Labor_Required,Num_Bricks,Tons_Iron,Tons_Sand,Total_Cost,Days
361,8,28.0,40,74960,16.0,70.00,2018760.0,49
73,6,21.0,30,37365,12.0,52.50,1400940.0,30
374,3,10.5,15,23065,6.0,26.25,726765.0,30
155,8,28.0,40,53246,16.0,70.00,1888476.0,35
104,6,21.0,30,67394,12.0,52.50,1581114.0,44
...,...,...,...,...,...,...,...,...
347,5,17.5,25,22480,10.0,43.75,1115505.0,30
86,1,5.0,5,30692,2.0,12.50,420402.0,30
75,7,24.5,35,40262,14.0,61.25,1614447.0,30
438,2,7.0,10,17747,4.0,17.50,498732.0,30


In [318]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, df_pred_2)*100
r2

97.10030773982